This notebook is associated with the paper "The relative class number one problem for function fields, I" by K.S. Kedlaya. It runs in SageMath (tested using version 9.4) and depends on Magma (tested using version
2.25-5); it also requires the pandas and openpyxl libraries (`sage --pip install openpyxl/pandas`).

In this notebook, we compute cyclic purely geometric extensions with $q=2$ using the upper bound on Weil polynomials. This depends on the exhaustion over Weil polynomials performed in the notebook "Weil polynomial bound for q=2" in this repository, the tabulation of curves of genus $\leq 4$ in LMFDB (including data by Xarles in genus 4), and data files of Dragutinović tabulating curves of genus 5. Allow 1.5 hours for completion.

In [1]:
load("../Shared/weil_poly_utils.sage")
load("../Shared/cyclic_covers.sage")

Read data about the exhaustion from an Excel spreadsheet. Note that we only use the data about base curves $C$, not covering curves $C'$ (except at the end to match up the results).

In [2]:
import pandas, itertools
from collections import defaultdict

In [3]:
df = pandas.read_excel('../Shared/polys.xlsx', index_col=int(0))

In [4]:
if "Unnamed: 0" in df:
    del df["Unnamed: 0"]
if "Cyclic" in df:
    del df["Cyclic"]
print(list(df))

['d', 'g', "g'", 'Label of J(C)', '#J(C)(F_2)', '#J(C)(F_4)', 'Counts of C', "Counts of C'"]


In [5]:
candidates = {}
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    if (d,g,g1) not in candidates:
        candidates[d,g,g1] = []
    s = r["Label of J(C)"]
    if s not in candidates[d,g,g1]:
        candidates[d,g,g1].append(s)

Table of curves with a given Weil polynomial, taken from LMFDB.

In [6]:
curves_by_poly = {}

In [7]:
P1.<x> = GF(2)[]
curves_by_poly[1] = {
    '1.2.ac': [(P1(1), x^3+x+1)],
    '1.2.ab': [(x, x^3+x^2+1)],
    '1.2.a': [(P1(1), x^3)],
    '1.2.b': [(x, x^3+1)],
    '1.2.c': [(P1(1), x^3+x)]
}

In [8]:
curves_by_poly[2] = {
    '2.2.ab_a': [(x,x^5+x^2+x)],
    '2.2.ab_c': [(x, x^5+x^3+x)],
    '2.2.ab_d': [(x^3+x+1, x^5+x^4+1)],
    '2.2.ac_e': [(P1(1), x^5+x^4+1)],
    '2.2.a_ab': [(x^2+x, x^5+x^3+x^2+x)],
    '2.2.a_a': [(P1(1), x^5)],
    '2.2.a_b': [(x^2+x+1, x^5+x^3+x^2+1)],
    '2.2.a_c': [(P1(1), x^5+x^4+x^3)],
    '2.2.b_a': [(x, x^5+x)],
    '2.2.b_b': [(x^3+x+1, x^6+x^3+x^2+x)],
    '2.2.b_c': [(x, x^5+x^3+x^2+x)],
    '2.2.b_d': [(x^3+x+1, x^6+x^5+x^4+x^2)],
    '2.2.c_c': [(P1(1), x^5+x^3)],
    '2.2.c_d': [(x^2+x+1, x^5+x^4+x^2+x)], 
    '2.2.c_e': [(P1(1),x^5+x^4)],
    '2.2.d_f': [(x^3+x+1,x^5+x^4+x^3+x)],
}

In [9]:
P3.<x0,x1,x2> = GF(2)[]
curves_by_poly[3] = {
    '3.2.ad_g_ai': [[(x^4+x^2+1,x^8+x+1)], []],
    '3.2.ac_c_ab': [[(x^4+x^2+x+1, x^8+x^7+x^5+x^3+x^2+1)],
                    [x0^4+x0^3*x1+x0^2*x2^2+x0*x2^3+x1^4+x1^3*x2+x1^2*x2^2]],
    '3.2.ac_e_af': [[(x^4+x^2+x,x^8+x^7+x^5+x^4+x)], []],
    '3.2.ab_a_a': [[(x, x^7+x^6+x), (x^4+x^2,x^8+x^5+x^3+x)],
                   [x0^4+x0^3*x1+x0^2*x1*x2+x0*x2^3+x1^4]],
    '3.2.ab_a_c': [[(x, x^7+x^6+x^5+x^2+x)], 
                   [x0^3*x1+x0^2*x1^2+x0^2*x1*x2+x0*x2^3+x1^4,
                    x0^3*x1+x0^2*x1^2+x0*x1^2*x2+x0*x2^3+x1^4]],
    '3.2.ab_a_e': [[], 
                   [x0^4+x0^3*x1+x0^3*x2+x0^2*x1*x2+x0*x1^3+x0*x1^2*x2+x0*x1*x2^2+x0*x2^3+x1^2*x2^2]],
    '3.2.ab_c_ac': [[(x, x^7+x^5+x), (x^4+x^2+1, x^5+x+1)],
                    [x0^4+x0^2*x1^2+x0^2*x1*x2+x0*x2^3+x1^4]],
    '3.2.ab_c_ab': [[], 
                    [x0^4+x0^3*x1+x0^2*x1*x2+x0*x2^3+x1^4+x1^3*x2+x1^2*x2^2]],
    '3.2.ab_c_a': [[(x^4+x^2,x^8+x)], []],
    '3.2.a_a_ab': [[(x^3+x+1, x^7+x^5+x^4+x^3+x+1)], []],
    '3.2.a_a_b': [[(x^3+x+1, x^7+x^6+x^5+x^4+x^3+x^2+x)],
                  [x0^4+x0^3*x2+x0^2*x1^2+x0^2*x1*x2+x0*x1*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.a_a_d': [[], [x0^4+x0^3*x1+x0^3*x2+x0^2*x1*x2+x0*x1^3+x0*x1^2*x2+x0*x2^3+x1^4,
                       x0^3*x2+x0^2*x1^2+x0^2*x1*x2+x0^2*x2^2+x0*x1*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.a_a_f': [[], [x0^3*x1+x0*x2^3+x1^3*x2]],
    '3.2.a_b_a': [[(x^2+x, x^7+x^5+x^2+x), (x^2+x+1, x^7+x^5+x^3+x^2+x+1)],
                  [x0^4+x0*x1^3+x0*x2^3+x1^4]],
    '3.2.a_b_d': [[], [x0^4+x0^3*x1+x0^3*x2+x0^2*x1^2+x0*x2^3+x1^3*x2, 
                       x0^4+x0^3*x1+x0^3*x2+x0^2*x1^2+x0*x2^3+x1^3*x2+x1^2*x2^2]],
    '3.2.a_c_ab': [[(x^4+x^2+x+1, x^8+x^7+x^4+x^2),
                    (x^4+x^2+x+1, x^6+x^5+x^2+1)],
                   [x0^4+x0^2*x1*x2+x0*x1^3+x0*x1^2*x2+x0*x2^3+x1^4]],
    '3.2.a_c_b': [[(x^4+x^2+x+1, x^7+1), (x^4+x^2+x+1, x^8+x^6+x^5+x^4)],
                  [x0^4+x0^3*x2+x0^2*x1*x2+x0*x1^3+x0*x1^2*x2+x0*x2^3+x1^2*x2^2]],
    '3.2.b_b_b': [[(x^4+x+1, x^8+x^7+x^5+x^3)],
                    [x0^3*x2+x0^2*x1*x2+x0*x1^3+x0*x2^3+x1^4]],
    '3.2.b_b_e': [[], [x0^3*x2+x0^2*x1^2+x0^2*x1*x2+x0*x2^3+x1^3*x2]],
    '3.2.b_c_a': [[(x^4+x^2,x^4+x)], []],
    '3.2.b_c_b': [[], [x0^3*x1+x0^2*x1*x2+x0*x2^3+x1^3*x2]],
    '3.2.b_c_c': [[(x,x^7+x^5+x^2+x), (x^4+x^2+1, x^8+x^5+x^4+x)],
                  [x0^4+x0^3*x1+x0^2*x1^2+x0*x1^2*x2+x0*x2^3+x1^4]],
    '3.2.b_c_d': [[], [x0^4+x0^3*x1+x0^2*x1^2+x0*x1*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.b_c_e': [[(x, x^7+x), (x^4+x^2, x^2+x), (x^4+x^2+1, x^8+x^4+x^2+x)],
                  [x0^4+x0^2*x1*x2+x0*x2^3+x1^4]],
    '3.2.b_d_c': [[], [x0^4+x0^3*x2+x0^2*x1^2+x0^2*x1*x2+x0^2*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.b_d_e': [[], [x0^4+x0^3*x2+x0*x1^3+x0*x1^2*x2+x0*x2^3+x1^2*x2^2]],
    '3.2.b_e_d': [[], [x0^4+x0^3*x1+x0^2*x1^2+x0^2*x1*x2+x0*x2^3+x1^3*x2]],
    '3.2.b_e_e': [[], [x0^4+x0^2*x1^2+x0*x1*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.c_d_d': [[], [x0^3*x1+x0^3*x2+x0*x2^3+x1^3*x2]],
    '3.2.c_d_f': [[], [x0^3*x1+x0^3*x2+x0^2*x1^2+x0^2*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.c_e_e': [[], [x0^3*x1+x0^3*x2+x0*x2^3+x1^4]],
    '3.2.c_e_f': [[(x^4+x^2+x, x^7+x^5+x^2+x)],
                    [x0^3*x1+x0^2*x1^2+x0^2*x2^2+x0*x2^3+x1^3*x2]],
    '3.2.c_e_g': [[(P1(1), x^7+x^4)], [x0^4+x0^2*x1^2+x0*x2^3+x1^3*x2]],
    '3.2.c_e_h': [[(x^4+x^2+x+1, x^5+x^3+x^2+x)], []],
    '3.2.d_g_i': [[(x^4+x^2+1, x^4+x)], []],
    '3.2.d_g_k': [[(x^4+x^2+1, x^5+x^2)], []],
    '3.2.d_h_l': [[(x^4+x+1,x^6+x^5+x^3+x)], []],
}

In [10]:
P4.<x0,x1,x2,x3> = GF(2)[]
curves_by_poly[4] = {
    '4.2.a_c_ab_c': [[(x^3+x+1, x^9+x^7+x^2)],
                     [(x0*x1+x3^2, x0^3+x1^3+x0*x1*x2+x2^3+x0*x1*x3+x0*x2*x3),
                      (x0*x1+x3^2, x0^3+x0^2*x1+x1^3+x0*x1*x2+x2^3+x0*x1*x3+x0*x2*x3)]],
    '4.2.a_c_ab_e': [[(x^3+x+1, x^9+x^6+x^2)], 
                     [(x0^2+x0*x1+x1^2+x2*x3, x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.a_c_ab_g': [[(x^3+x+1, x^9+x^7+x+1)],
                     [(x0*x1+x3^2, x1^3+x0*x1*x2+x2^3+x0^2*x3+x0*x1*x3+x0*x2*x3),
                      (x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x1^3+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.a_c_b_c': [[(x^3+x+1, x^9+x^7+x^6+1)],
                    [(x0*x1+x2*x3, x0^3+x1^3+x0^2*x2+x1^2*x2+x0*x2^2+x1*x2^2+x0*x3^2),
                     (x0*x1+x2*x3, x0*x1^2+x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x2^3+x0^2*x3+x0*x3^2),
                     (x0*x1+x3^2, x0^3+x1^3+x0*x1*x2+x1^2*x2+x2^3+x0*x2*x3),
                     (x0^2+x0*x1+x1^2+x2*x3, x0^2*x1+x0*x1^2+x1*x2^2+x0^2*x3+x0*x3^2)]],
    '4.2.a_c_b_e': [[(x^3+x+1, x^9+1)],
                    [(x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x0*x1^2+x1^2*x2+x0*x2^2+x0^2*x3+x0*x3^2),
                     (x0^2+x0*x1+x1^2+x2*x3, x0^2*x1+x0*x1^2+x0^2*x2+x1*x2^2+x0^2*x3+x0*x3^2)]],
    '4.2.a_c_b_g': [[(x^3+x+1, x^9+x^7+x^6+x^2+x)],
                    [(x0^2+x0*x1+x1^2+x2*x3, x1^3+x0^2*x2+x0*x2^2+x0*x3^2)]],
    '4.2.a_c_d_a': [[], [(x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x0*x1^2+x1*x2^2+x0^2*x3+x0*x3^2)]],
    '4.2.a_c_d_c': [[], [(x0^2+x0*x1+x1^2+x2*x3, x0^3+x1^3+x0^2*x2+x0*x1*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.a_d_ab_f': [[], [(x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x0^2*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.a_d_ab_h': [[], [(x0*x1+x3^2, x0^3+x0^2*x1+x0*x1^2+x1^3+x0*x1*x2+x2^3+x1^2*x3+x0*x2*x3)]],
    '4.2.a_d_b_f': [[],
                    [(x0*x1+x2*x3, x0^3+x0*x1^2+x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x1*x2^2+x0*x3^2),
                     (x0*x1+x3^2, x0^3+x0*x1*x2+x2^3+x0^2*x3+x1^2*x3+x0*x2*x3),
                     (x0*x1+x3^2, x0^3+x1^3+x0*x1*x2+x2^3+x1^2*x3+x0*x2*x3),
                     (x0*x1+x3^2, x0^3+x0^2*x1+x1^3+x0*x1*x2+x2^3+x1^2*x3+x0*x2*x3),
                     (x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x0*x2^2+x0*x3^2)]],
    '4.2.a_d_b_h': [[], [(x0*x1+x2*x3, x0^3+x0*x1^2+x1^3+x0^2*x2+x1^2*x2+x0*x2^2+x0^2*x3+x0*x1*x3+x0*x3^2),
                          (x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2),
                          (x0^2+x0*x1+x1^2+x2*x3, x0^3+x1^3+x0*x1*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.a_d_d_f': [[], [(x0*x1+x3^2, x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x2^3+x1^2*x3+x0*x2*x3)]],
    '4.2.a_e_b_k': [[], [(x0*x1+x3^2, x0^3+x0^2*x1+x1^3+x0*x1*x2+x2^3+x0^2*x3+x0*x1*x3+x0*x2*x3)]],
    '4.2.b_b_a_b': [[], [(x0*x1+x3^2, x0^2*x1+x1^2*x2+x2^3+x0^2*x3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_b_a_d': [[], [(x0*x1+x2*x3, x0^3+x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.b_b_c_d': [[], [(x0*x1+x2*x3, x0*x1^2+x1^3+x0^2*x2+x0*x1*x2+x0*x2^2+x0*x1*x3+x0*x3^2),
                         (x0*x1+x3^2, x0^2*x2+x1^2*x2+x2^3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_b_c_f': [[], [(x0*x1+x2*x3, x0^2*x1+x0*x1^2+x1^3+x0^2*x2+x0*x1*x2+x0*x2^2+x0*x3^2),
                         (x0*x1+x3^2, x0^3+x0^2*x1+x0^2*x2+x1^2*x2+x2^3+x0^2*x3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_b_c_h': [[], [(x0*x1+x3^2, x0^3+x1^2*x2+x2^3+x1^2*x3+x0*x2*x3),
                         (x0^2+x0*x1+x1^2+x2*x3, x1^3+x0*x1*x2+x0*x2^2+x0*x3^2)]],
    '4.2.b_b_a_b': [[], [(x0*x1+x3^2, x0^2*x1+x1^2*x2+x2^3+x0^2*x3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_b_a_d': [[], [(x0*x1+x2*x3, x0^3+x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.b_c_a_a': [[(x^2, x^9+x^7+x+1)], []],
    '4.2.b_c_a_c': [[(x, x^9+x^7+x^5+x^3+x^2+1)], []],
    '4.2.b_c_a_e': [[], [(x0^2+x0*x1+x1^2+x2*x3, x0*x1*x2+x1^2*x2+x0*x2^2+x0*x3^2)]],
    '4.2.b_c_c_c': [[(x, x^9+x^7+x^3+1)], []],
    '4.2.b_c_c_e': [[(x^2, x^9+x^5+x^4+x), (x^2, x^9+x^4+x+1)], []],
    '4.2.b_c_c_g': [[(x, x^9+1)],
                    [(x0*x1+x2*x3, x0^3+x0*x1^2+x0^2*x2+x1^2*x2+x0*x2^2+x1^2*x3+x0*x3^2),
                     (x0*x1+x2*x3, x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x1*x2^2+x0^2*x3+x0*x3^2),
                     (x0*x1+x3^2, x1^3+x0^2*x2+x0*x1*x2+x2^3+x0^2*x3),
                     (x0^2+x0*x1+x1^2+x2*x3, x0^3+x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.b_b_c_h': [[], [(x0*x1+x3^2, x0^3+x1^2*x2+x2^3+x1^2*x3+x0*x2*x3),
                         (x0^2+x0*x1+x1^2+x2*x3, x1^3+x0*x1*x2+x0*x2^2+x0*x3^2)]],
    '4.2.b_c_d_h': [[(x^4+x^2+x, x^9+x^8+x^2+1)], 
                    [(x0*x1+x2*x3, x0^3+x0*x1^2+x1^3+x0*x1*x2+x0*x2^2+x0*x1*x3+x0*x3^2),
                     (x0^2+x0*x1+x1^2+x2*x3, x0^2*x1+x1^3+x0*x1*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.b_c_d_j': [[], [(x0*x1+x3^2, x0^3+x0^2*x1+x2^3+x0^2*x3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_c_e_e': [[(x^2, x^9+x^7+x^5+x)], []],
    '4.2.b_d_c_h': [[], [(x0*x1+x2*x3, x0^3+x0^2*x1+x0*x1^2+x0*x1*x2+x1^2*x2+x0*x2^2+x0^2*x3+x1^2*x3+x0*x3^2),
                         (x0*x1+x2*x3, x0^3+x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x0^2*x3+x1^2*x3+x0*x3^2),
                         (x0*x1+x3^2, x0^3+x0^2*x1+x0*x1^2+x1^3+x2^3+x0*x1*x3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_d_e_j': [[], [(x0*x1+x3^2, x0^3+x1^3+x2^3+x0*x1*x3+x1^2*x3+x0*x2*x3)]],
    '4.2.b_e_c_i': [[], [(x0*x1+x3^2, x0^3+x1^3+x2^3+x0*x2*x3)]],
    '4.2.b_e_e_k': [[], [(x0*x1+x3^2, x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x2^3+x0^2*x3),
                         (x0^2+x0*x1+x1^2+x2*x3, x0^2*x1+x1^3+x0^2*x2+x0*x1*x2+x1^2*x2+x0*x2^2+x0*x1*x3+x0*x3^2)]],
    '4.2.c_e_h_k': [[(x^3+x+1, x^9+x^7)], []],
    '4.2.c_g_i_q': [[], 
                    [(x0*x1+x2*x3,x0^3+x0*x1^2+x0^2*x2+x1^2*x2+x0*x2^2+x1*x2^2+x0^2*x3+x0*x1*x3+x1^2*x3+x0*x3^2),
                     (x0*x1+x2*x3,x1^3+x0*x1*x2+x1^2*x2+x0*x2^2+x1*x2^2+x0^2*x3+x0*x1*x3+x1^2*x3+x0*x3^2)]],
    '4.2.d_f_i_n': [[], [(x0*x1+x2*x3, x0^2*x1+x0*x1^2+x0^2*x2+x0*x1*x2+x1^2*x2+x1*x2^2+x0^2*x3+x1^2*x3+x0*x3^2)]],
    '4.2.d_f_k_s': [[], 
                    [(x0*x1+x3^2, x0^2*x1+x0^2*x2+x0*x1*x2+x1^2*x2+x2^3+x0^2*x3+x0*x1*x3+x1^2*x3)]],
    '4.2.d_h_l_r': [[(x^5+x^2+1, x^4+x^2)], 
                    [(x0*x1+x2*x3,x0^2*x1+x0*x1^2+x1^3+x0*x1*x2+x1^2*x2+x0*x2^2+x0^2*x3+x1^2*x3+x0*x3^2)]],
    '4.2.d_i_o_x': [[],
                    [(x0*x1+x2*x3,x0^2*x2+x0*x1*x2+x0*x2^2+x1*x2^2+x0^2*x3+x1^2*x3+x0*x3^2)]],
    '4.2.e_j_q_z': [[], [(x0*x1+x2*x3, x0^2*x1+x1^3+x0*x1*x2+x1^2*x2+x0*x2^2+x0^2*x3+x1^2*x3+x0*x3^2)]],
    '4.2.e_k_u_bg': [[], [(x0*x1+x2*x3, x0^2*x2+x1^2*x2+x1*x2^2+x0^2*x3+x1^2*x3+x0*x3^2)]],
    '4.2.f_o_bc_bs': [[], [(x0*x1+x2*x3, x0*x1^2+x1^3+x0^2*x2+x1^2*x2+x0*x2^2+x0^2*x3+x1^2*x3+x0*x3^2)]]
}

Populate the table in genus 5 using data from Dragutinović.

In [11]:
curves_by_poly[5] = {}
with open("HyperellipticCurvesData.txt", "r") as f:
    s = f.read()
    l = sage_eval(s, locals={'x':P1.gen()})
for (h, g) in l:
    C = HyperellipticCurve(g,h)
    assert C.genus() == 5
    u = C.frobenius_polynomial()
    s = label_from_weil_poly(u)
    if s not in curves_by_poly[5]: 
        curves_by_poly[5][s] = [[],[],[]]
    curves_by_poly[5][s][0].append((h,g))

In [12]:
with open("TrigonalCurvesWithAutomorphisms.txt", "r") as f:
    s = f.read()
    l = sage_eval(s)
monos5 = [prod(t) for t in itertools.combinations_with_replacement(P3.gens(), 5)]
proj = magma.ProjectiveSpace(P3)
for (v, _) in l:
    gen = sum(v[i]*monos5[i] for i in range(len(monos5)))
    X = proj.Scheme([gen])
    assert X.Dimension() == 1
    C = X.Curve()
    assert C.Genus() == 5
    ct = tuple(Integer(C.NumberOfPlacesOfDegreeOneECF(i)) for i in range(1, 6))
    u = weil_poly_from_point_count(ct, 5)
    s = label_from_weil_poly(u)
    if s not in curves_by_poly[5]: 
        curves_by_poly[5][s] = [[],[],[]]
    curves_by_poly[5][s][1].append(gen)

In [13]:
with open("Complete_Intersections.txt", "r") as f:
    s = f.read()
    l1 = sage_eval(s)
with open("Pts_Count_Complete_Intersections.txt", "r") as f:
    s = f.read()
    l2 = sage_eval(s)
P5.<x0,x1,x2,x3,x4> = GF(2)[]
monos2 = [prod(t) for t in itertools.combinations_with_replacement(P5.gens(), 2)]
for i in range(len(l1)):
    gens = [sum(v[j]*monos2[j] for j in range(len(monos2))) for v in l1[i]]
    u = weil_poly_from_point_count(l2[i], 5)
    s = label_from_weil_poly(u)
    if s not in curves_by_poly[5]: 
        curves_by_poly[5][s] = [[],[],[]]
    curves_by_poly[5][s][2].append(gens)

Produce Magma function fields of curves with a given zeta function.

In [14]:
def function_fields_by_weil_poly(s, m=1):
    if m == 1:
        F0 = magma.FunctionField(GF(2))
    else:
        K.<a> = GF(2^m)
        F0 = magma.FunctionField(K)
    F0.AssignNames('x')
    S = magma.PolynomialRing(F0)
    S.AssignNames('y')
    l = s.split(".")
    g = Integer(l[0])
    if s in curves_by_poly[g]:
        ans = []
        # Hyperelliptics.
        m = curves_by_poly[g][s] if g <= 2 else curves_by_poly[g][s][0]
        for (u,v) in m:
            yield magma.HyperellipticCurve(v,u).FunctionField().AlgorithmicFunctionField()
        # Plane quartics for g=3.
        if g == 3:
            proj = magma.ProjectiveSpace(P3)
            for u in curves_by_poly[g][s][1]:
                yield proj.Scheme([u]).FunctionField().AlgorithmicFunctionField()
        # Genus 4 curves.
        if g == 4:
            proj = magma.ProjectiveSpace(P4)
            for (u,v) in curves_by_poly[g][s][1]:
                yield proj.Scheme([u,v]).FunctionField().AlgorithmicFunctionField()
        # Genus 5 curves.
        if g == 5:
            proj = magma.ProjectiveSpace(P3)
            for u in curves_by_poly[g][s][1]:
                yield proj.Scheme([u]).FunctionField().AlgorithmicFunctionField()        
            proj = magma.ProjectiveSpace(P5)        
            for (u,v,w) in curves_by_poly[g][s][2]:
                yield proj.Scheme([u,v,w]).FunctionField().AlgorithmicFunctionField()

Check that the zeta functions of these curves are as indicated.

In [15]:
P.<T> = QQ[]
for g in range(1, 6):
    for s in curves_by_poly[g]:
        for F in function_fields_by_weil_poly(s):
            u = F.ZetaFunction().Numerator().sage()(T).reverse()
            assert label_from_weil_poly(u) == s

Check the genus 5 data for cases which can occur as a cover of a genus-1 curve with relative class number 1.

In [16]:
targets = [[0,0,9,12,50],
           [0,0,12,16,60],
           [0,2,6,18,30],
           [0,2,3,42,30],
           [0,4,0,16,20],
           [0,4,0,28,25],
           [0,6,0,18,25],
           [1,1,1,25,1]]
ans = []
for t in targets:
    u = weil_poly_from_point_count(t, 5)
    s = label_from_weil_poly(u)
    if s in curves_by_poly[5]:
        ans.append(s)
print(ans)

['5.2.ad_c_d_ag_h', '5.2.ad_c_e_ai_i', '5.2.ad_e_ag_k_ao']


Construct extensions with relative class number 1.

In [17]:
covers_by_poly = {}

In [18]:
def match_weil_poly_general(s, d, delta=0, verbose=False):
    l = s.split(".")
    g = Integer(l[0])
    g1 = g + (d-1)*(g-1) + delta
    ans = []
    for F in function_fields_by_weil_poly(s):
        ans.append((s, match_weil_poly(F, d, delta=delta, verbose=verbose)))
    if ans:
        if (d, g, g1) not in covers_by_poly:
            covers_by_poly[d, g, g1] = {}
        if s not in covers_by_poly[d, g, g1]:
            covers_by_poly[d, g, g1][s] = {}
        for (_, l) in ans:
            for (F, F1) in l:
                if F not in covers_by_poly[d, g, g1][s]:
                    covers_by_poly[d, g, g1][s][F] = []
                if F1 not in covers_by_poly[d, g, g1][s][F]:
                    covers_by_poly[d, g, g1][s][F].append(F1)
    return ans

In [19]:
for (d, g, g1) in candidates:
    print(d, g, g1)
    delta = g1-g-(d-1)*(g-1)
    if g <= 5:
        covers_by_poly[d, g, g1] = {}
        for s in candidates[d,g,g1]:
            match_weil_poly_general(s, d, delta=g1-g-(d-1)*(g-1))

2 2 3
2 2 4
2 2 5
2 3 5
2 3 6
2 4 7
2 4 8
2 5 9
2 6 11
2 7 13
3 2 4
3 2 6
3 3 7
3 4 10
4 2 5
4 3 9
5 2 6
6 2 7
7 2 8


List isomorphism class representatives for the base fields.

In [20]:
for (d, g, g1) in covers_by_poly:
    print(d,g,g1)
    for s in covers_by_poly[d,g,g1]:
        if covers_by_poly[d,g,g1][s]:
            l = isomorphism_class_reps(list(covers_by_poly[d,g,g1][s].keys()))
            print(s, l)

2 2 3
2.2.ab_c [Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + x*y + x^5 + x^3 + x]
2.2.b_c [Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + x*y + x^5 + x^3 + x^2 + x]
2 2 4
2.2.a_a [Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + y + x^5]
2.2.a_c [Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + y + x^5 + x^4 + x^3]
2.2.b_b [Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + (x^3 + x + 1)*y + x^6 + x^3 + x^2 + x]
2 2 5
2.2.b_d [Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + (x^3 + x + 1)*y + x^6 + x^5 + x^4 + x^2]
2.2.c_e [Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + y + x^5 + x^4]
2 3 5
3.2.ad_g_ai [Algebraic function field defined over Univariate rational function field ov

4.2.e_j_q_z [Algebraic function field defined over Univariate rational function field over GF(2) by
x*y^3 + (x^2 + x + 1)*y^2 + (x^4 + x)*y + x^5 + x^4]
4 2 5
2.2.ab_c [Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + x*y + x^5 + x^3 + x]
4 3 9
5 2 6
2.2.a_a [Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + y + x^5]
2.2.b_c [Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + x*y + x^5 + x^3 + x^2 + x]
2.2.c_e [Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + y + x^5 + x^4]
6 2 7
7 2 8
2.2.c_d [Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + (x^2 + x + 1)*y + x^5 + x^4 + x^2 + x]


Check that no covers that occur for $d=4, g=2, g'=5$ are dihedral, by comparing options for the intermediate covers with $(d,g,g') = (2,2,3)$ and $(d,g,g') = (2,3,5)$.

In [21]:
covers_by_poly[2,2,3]['2.2.ab_c']

{Algebraic function field defined over Univariate rational function field over GF(2) by
 y^2 + x*y + x^5 + x^3 + x: [Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
  y^2 + x*y + x^5 + x^3 + x by
  $.1^2 + $.1 + x^3 + x + 1]}

In [22]:
covers_by_poly[2,3,5]['3.2.ad_g_ai']

{Algebraic function field defined over Univariate rational function field over GF(2) by
 y^2 + (x^4 + x^2 + 1)*y + x^8 + x + 1: [Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
  y^2 + (x^4 + x^2 + 1)*y + x^8 + x + 1 by
  $.1^2 + $.1 + $.1 + x^2 + 1,
  Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
  y^2 + (x^4 + x^2 + 1)*y + x^8 + x + 1 by
  $.1^2 + $.1 + $.1 + x^2]}

In [23]:
covers_by_poly[4,2,5]['2.2.ab_c']

{Algebraic function field defined over Univariate rational function field over GF(2) by
 y^2 + x*y + x^5 + x^3 + x: [Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
  y^2 + x*y + x^5 + x^3 + x by
  $.1^4 + (x^3 + x)*$.1^2 + (x^3 + x + 1)*$.1 + (x^4 + x)*$.1 + x^10 + x^8 + x^7 + x^3 + x^2 + x + 1,
  Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
  y^2 + x*y + x^5 + x^3 + x by
  $.1^4 + (x^3 + x)*$.1^2 + (x^3 + x + 1)*$.1 + (x^4 + x)*$.1 + x^10 + x^8 + x^7 + x^2 + 1]}

In [24]:
for _, l1 in covers_by_poly[2,3,5]['3.2.ad_g_ai'].items():
    for _, l2 in covers_by_poly[4,2,5]['2.2.ab_c'].items():
        for F1 in l1:
            for F2 in l2:
                print(F1.IsIsomorphic(F2))

true
true
true
true


Identify base curves $C$ when $(d,g,g') = (2,7,13)$ and $\#C(\mathbb{F}_2) = 6$.

In [25]:
P.<T> = QQ[]
u0 = T^6 + 3*T^5 - 3*T^4 - 11*T^3 + 3*T^2 + 9*T - 4
l1 = []
for F in function_fields_by_weil_poly('1.2.a'):
    places3 = [i for i in F.Places(3)]
    z = F.DivisorGroup().Identity()
    M = z + 2*places3[0] + 2*places3[1]
    for F1 in cyclic_covers_by_ramification(F, 2, M, delta=6):
        s = magma_poly_list(F1.ZetaFunction().Numerator() // F.ZetaFunction().Numerator())
        u = P(s).reverse()
        u1 = u.trace_polynomial()[0]
        if u1 == u0:
            l1 += match_weil_poly(F1, 2)

In [26]:
print(l1)

[(Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + y + x^3 by
$.1^2 + $.1 + x^2/(x^3 + x^2 + 1), Algebraic function field defined over Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + y + x^3 by
$.1^2 + $.1 + x^2/(x^3 + x^2 + 1) by
$.1^2 + $.1 + ((x^6 + x^4 + x^3 + x^2)*$.1 + (x^9 + x^7 + x^4 + x))*w + (x^5 + x^2 + x)*$.1 + x^8 + x^2 + x + 1)]


In [27]:
covers_by_poly[2, 7, 13] = {}
for (F, F1) in l1:
    u = P(magma_poly_list(F.ZetaFunction().Numerator())).reverse()
    s = label_from_weil_poly(u)
    if s not in covers_by_poly[2, 7, 13]:
        covers_by_poly[2, 7, 13][s] = {}
    if F not in covers_by_poly[2, 7, 13][s]:
        covers_by_poly[2, 7, 13][s][F] = []
    if F1 not in covers_by_poly[2, 7, 13][s][F]:
        covers_by_poly[2, 7, 13][s][F].append(F1)

Convert the covers into numerical data in preparation for writing out to the spreadsheet.

In [28]:
l = []
for (d, g, g1) in candidates:
    print(d, g, g1)
    if (d, g, g1) in covers_by_poly:
        for s in covers_by_poly[d, g, g1]:
            for F in covers_by_poly[d, g, g1][s]:
                for F1 in covers_by_poly[d, g, g1][s][F]:
                    u1 = T.parent()(magma_poly_list(F.ZetaFunction().Numerator())).reverse()
                    v1 = T.parent()(magma_poly_list(F1.ZetaFunction().Numerator())).reverse()
                    l.append((d, g, g1, point_count_from_weil_poly(u1, 13),
                              point_count_from_weil_poly(v1, 13)))

2 2 3
2 2 4
2 2 5
2 3 5
2 3 6
2 4 7
2 4 8
2 5 9
2 6 11
2 7 13
3 2 4
3 2 6
3 3 7
3 4 10
4 2 5
4 3 9
5 2 6
6 2 7
7 2 8


Write the results back to the spreadsheet. In the process, we check that every covering we found gives a pair of Weil polynomials from our original list.

In [29]:
df['Cyclic'] = 'No'
used_pols = []
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    ct1 = eval(r["Counts of C"])
    ct2 = eval(r["Counts of C'"])
    if (d, g, g1, ct1, ct2) in l:
        df.loc[i, "Cyclic"] = "Yes"
        used_pols.append((d, g, g1, ct1, ct2))
    elif d == 2 and (g == 6 or g == 7):
        df.loc[i, "Cyclic"] = "Unknown"
assert all(t in used_pols for t in l)

In [30]:
df.to_excel('../Shared/polys.xlsx', sheet_name='Weil polynomials', merge_cells=True, freeze_panes=(int(1),int(1)))